In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets

# Random Forest y ajuste de parámetros

Realizaremos este ejercicio con un conjunto de datos relacionados con la salud pública.

Como es lógico vamos a repetir el proceso que hemos seguido hasta el momento con las otras técnicas de clasificación:

* Exploración del conjunto de datos.
* Tratamiento de las características.
* Separación en conjuntos de entrenamiento y evaluación.
* Aplicación del método.
* Análisis de los resultados.

En esta práctica vamos a analizar cómo podemos ajustar los parámetros de un clasificador sin tener que realizar la búsqueda de manera manual. 

[Documentación de random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [2]:
data= sklearn.datasets.load_breast_cancer()
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [3]:
data.keys()
X = data.data
y = data.target

**Separación del conjunto de entrenamiento y del conjunto de evaluación**

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [19]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=5)
# Fit on training data
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

**Evaluación de los resultados**

In [18]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Matriz de confusión")
print(confusion_matrix(y_test, y_pred))
print("-_"*50)
print(classification_report(y_test,y_pred))
print("-_"*50)
print("Accuracy (TP + TN) / Total: %.2f" %(accuracy_score(y_test, y_pred)))

Matriz de confusión
[[39  4]
 [ 3 68]]
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        43
           1       0.94      0.96      0.95        71

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Accuracy (TP + TN) / Total: 0.94


#### Búsqueda de los mejores parámetros.

Esta técnica, presenta una capacidad mayor de parametrizar. Vamos a tratar con los siguientes:

* Número de árboles  ```n_estimators``` .
* Profundidad máxima de los árboles ```max_depth```.
* Características máximas: la cantidad máxima de características que el clasificador puede probar en cada árbol ```max_features```.
* Observaciones mínimas: determina el número mínimo de observaciones necesarias para dividir un nodo interno. ```min_sample_leaf```.
* Bootstrapping: Parámetro booleano ```bootstrap```.

Vamos a usar una búsqueda exhaustiva sobre estos parámetros usando la clase ```GridSearchCV ```[Documentación](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV).

Si notais que la búsqueda exhaustiva es muy lenta, podeis usar la búsqueda aleatoria ```RandomizedSearchCV``` [Documentación](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV).

In [7]:
from sklearn.model_selection import GridSearchCV# Number of trees in random forest
n_estimators = [1,5,10,20,50,100,200,250,500,1000]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
param_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               #'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [8]:
rfc = RandomForestClassifier()

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
print(CV_rfc.best_params_)

{'bootstrap': True, 'max_features': 'auto', 'n_estimators': 100}


In [9]:
y_pred = CV_rfc.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Matriz de confusión")
print(confusion_matrix(y_test, y_pred))
print("-_"*50)
print(classification_report(y_test,y_pred))
print("-_"*50)
print("Accuracy (TP + TN) / Total: %.2f" %(accuracy_score(y_test, y_pred)))

Matriz de confusión
[[38  5]
 [ 3 68]]
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        43
           1       0.93      0.96      0.94        71

    accuracy                           0.93       114
   macro avg       0.93      0.92      0.92       114
weighted avg       0.93      0.93      0.93       114

-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Accuracy (TP + TN) / Total: 0.93


### Extra, extra!!!

Para realizar una regresión scikit tiene la clase [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).

Usala para con el conjunto de datos de prueba de Boston.